In [6]:
# define some input data locations

from os.path import join
path = '/home/nils/data/EUR-44'
files = ['tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19710101-19751231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19760101-19801231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19810101-19851231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19860101-19901231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19910101-19951231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19960101-20001231.nc',]
resource = [ join(path,nc) for nc in files ]
resource

['/home/nils/data/EUR-44/tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19710101-19751231.nc',
 '/home/nils/data/EUR-44/tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19760101-19801231.nc',
 '/home/nils/data/EUR-44/tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19810101-19851231.nc',
 '/home/nils/data/EUR-44/tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19860101-19901231.nc',
 '/home/nils/data/EUR-44/tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19910101-19951231.nc',
 '/home/nils/data/EUR-44/tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19960101-20001231.nc']

Execution with plain python (no birdhouse components included )

In [2]:
import icclim 

from ocgis import RequestDataset, OcgOperations, env
from ocgis.contrib.library_icclim import IcclimTG90p

env.OVERWRITE = True
########################################################################################################################
# Compute a custom percentile basis using ICCLIM. ######################################################################
########################################################################################################################

# Subset the input dataset to return the desired base period for the percentile basis.
variable = 'tas'
years = range(1971, 2001)
time_region = {'year': years}

rd = RequestDataset(uri=resource, variable=variable)
field = rd.get()
field.get_time_region(time_region)

# Calculate the percentile basis. The data values must be a three-dimensional array.
arr = field.variables[variable].value.squeeze()
dt_arr = field.temporal.value_datetime
percentile = 90
window_width = 5
percentile_dict = IcclimTG90p.get_percentile_dict(arr, dt_arr, percentile, window_width)

calc = [{'func': 'icclim_TG90p', 'name': 'TG90p', 'kwds': {'percentile_dict': percentile_dict}}]
calc_grouping = 'year'
ops = OcgOperations(dataset=rd, calc=calc, calc_grouping=calc_grouping, output_format='nc')
coll = ops.execute()
print coll

/home/nils/birdhouse/flyingpigeon/notebooks/ocgis_output/ocgis_output.nc


Execute WPS icclim processes with PHYTHON 

In [2]:
#############################
# load the required libraries 
#############################

from owslib.wps import WebProcessingService, monitorExecution, printInputOutput

#################################################
# connect to the compute provider hosting the WPS
#################################################

# wps_url = "http://birdhouse-lsce.extra.cea.fr:8093/wps"
wps_url = "http://localhost:8093/wps"
wps = WebProcessingService(url=wps_url, verbose=False)

Explore some informations:

In [3]:
wps.identification.title

'Flyingpigeon'

In [4]:
for process in wps.processes:
    print '%s \t : %s '% (process.identifier, process.abstract) 

subset_continents 	 : Returns only the selected polygon for each input dataset 
subset_countries 	 : Returns only the selected polygon for each input dataset 
subset_regionseurope 	 : Returns only the selected polygon for each input dataset 
subset_points 	 : Extract Timeseries for specified coordinates from gridded datasets 
indices_single 	 : Climate indices based on one single input variable. 
indices_percentile 	 : Climate indices based on one single input variable and the percentile of a referece period. 
weatherregimes_reanalyse 	 : Weather Regimes based on pressure patterns, fetching selected Realayses Datasets 
weatherregimes_model 	 : Weather Regimes based on pressure patterns, fetching selected Realayses Datasets 
weatherregimes_projection 	 : Weather Regimes detection based on trained reference statistics 
analogs_detection 	 : Search for days with analog pressure pattern 
analogs_viewer 	 : Visualisation of text output of analogue process 
segetalflora 	 : Species biodivers

In [5]:
execute = wps.execute(
    identifier="indices_percentile",
    inputs=[
    ('indices','TG_p'),
    ('groupings','yr'),
    ('percentile', '95'),    
    ('resource',resource[0]),
    ('resource',resource[1]),
    ('resource',resource[2]),
    ('resource',resource[3]),
    ('resource',resource[4]),
    ('resource',resource[5]),
    ('polygons', 'DEU')    
    ], async=True
    )


 owslib.wps.WPSException : {'locator': 'None', 'code': 'NoApplicableCode', 'text': "[Errno 2] No such file or directory: './pywpsInputJSenKs.base64'"}


IndexError: list index out of range

In [7]:
# check process if completed ...

monitorExecution(execute, sleepSecs=5)
print execute.getStatus()

for o in execute.processOutputs:
    print o.reference

ProcessFailed
